In [11]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/552159c4da984fa5e56eda310634b92d098b9cf8/minsearch.py

--2025-09-09 04:55:26--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/552159c4da984fa5e56eda310634b92d098b9cf8/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4273 (4.2K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   4.17K  --.-KB/s    in 0s      

2025-09-09 04:55:26 (53.7 MB/s) - ‘minsearch.py.2’ saved [4273/4273]



In [18]:
import minsearch as ms

/workspaces/llm-zoomcamp/01-intro/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [12]:
import json

In [13]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [14]:
documents =[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [16]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [19]:
index = ms.Index(text_fields=["question", "text", "section"],
    keyword_fields=["course"])

In [21]:
q = "Can i still resume the course even if it had already started"

In [20]:
index.fit(documents)

In [24]:
boost = {'question': 3.0, 'section': 0.5}
result = index.search(
    query = q,
    boost_dict = boost,
    num_results = 5,
    filter_dict = {'course': 'data-engineering-zoomcamp'}
)

In [25]:
result

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, the slack channel remains open and you can ask questions there. But always sDocker containers exit code w search the channel first an